In [ ]:
from datasets import load_dataset
import numpy as np
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer, DefaultDataCollator, ViTConfig

import os
import torch
import torchvision.models as models # for CNN models
import torch.nn as nn # Added for neural network layers
!pip install -q evaluate # Ensure evaluate is installed
!pip install -q  accelerate transformers evaluate



# 1. Gerekli Kütüphanelerin Kurulumu
!pip install -q datasets transformers evaluate accelerate torch torchvision

# 2. Import İşlemleri
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Cihaz kontrolü (GPU var mı?)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")

In [ ]:
# 3. Veri Setini Yükleme
print("Veri seti Hugging Face Hub üzerinden indiriliyor...")
dataset = load_dataset("AiresPucrs/chest-xray")

# 4. Veri Seti Yapısını İnceleme
print("\n--- Veri Seti Genel Yapısı ---")
print(dataset)

# Sınıf etiketlerini kontrol edelim
# Dokümana göre: 0 -> NORMAL, 1 -> PNEUMONIA olmalı [cite: 13]
print(f"\nÖzellikler: {dataset['train'].features}")

In [ ]:
# Veri setinin bölümlerini kontrol et
if 'test' not in dataset:
    print("\nVeri seti hazır bölünmüş gelmedi. %80-%20 olarak biz bölüyoruz...")
    # Veriyi karıştırıp bölüyoruz (Seed sabitliyoruz ki sonuç değişmesin)
    ds_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
    train_ds = ds_split['train']
    test_ds = ds_split['test']
else:
    print("\nVeri seti hazır bölünmüş geldi, kontrol ediliyor...")
    train_ds = dataset['train']
    test_ds = dataset['test']

# Sayıları raporlayalım
print(f"\nEğitim Verisi Sayısı: {len(train_ds)}")
print(f"Test Verisi Sayısı: {len(test_ds)}")

total = len(train_ds) + len(test_ds)
print(f"Gerçekleşen Oran -> Eğitim: %{len(train_ds)/total*100:.1f}, Test: %{len(test_ds)/total*100:.1f}")

In [ ]:
# Rastgele birkaç görüntü çizdirelim
def show_samples(ds, num_samples=4):
    plt.figure(figsize=(12, 4))
    for i in range(num_samples):
        idx = np.random.randint(0, len(ds))
        image = ds[idx]['image']
        label = ds[idx]['label']

        # Label ismini bulma (features'dan)
        label_name = ds.features['label'].int2str(label)

        plt.subplot(1, num_samples, i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f"{label_name} ({label})")
        plt.axis('off')
    plt.show()

print("\n--- Eğitim Verisinden Örnekler ---")
show_samples(train_ds)

In [ ]:
# 1. Dönüşüm (Transform) Tanımları
# Görüntüleri 224x224 yapacağız (ViT ve ResNet standart girişi)
# RGB'ye çeviriyoruz ki ilerideki modellerde kanal sayısı hatası almayalım.
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 2. Dönüşümü Veri Setine Uygulama Fonksiyonu
def apply_transforms(examples):
    # Görüntülerin hepsi RGB olmayabilir, garantiye alıyoruz (.convert("RGB"))
    examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["image"]]
    return examples

# Veri setine dönüşümleri anlık (on-the-fly) uygulanacak şekilde atıyoruz
train_ds.set_transform(apply_transforms)
test_ds.set_transform(apply_transforms)

# 3. DataLoader için Collate Fonksiyonu
# Verileri batch (paket) haline getirirken Tensor yığınına (stack) çevirir
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return pixel_values, labels

# 4. DataLoader Oluşturma
BATCH_SIZE = 32

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Kontrol edelim
batch_img, batch_label = next(iter(train_loader))
print(f"Batch Görüntü Boyutu: {batch_img.shape}") # Beklenen: [32, 3, 224, 224]
print(f"Batch Etiket Boyutu: {batch_label.shape}") # Beklenen: [32]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()

        # --- 1. Konvolüsyon Bloğu ---
        # Giriş: 3 kanal (RGB), Çıkış: 32 filtre
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32) # BatchNorm
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Pooling

        # --- 2. Konvolüsyon Bloğu ---
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # --- 3. Konvolüsyon Bloğu ---
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # --- 4. Konvolüsyon Bloğu (Opsiyonel, derinliği artırmak için) ---
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        # --- Sınıflandırma Katmanı (Dense/Linear) ---
        self.flatten = nn.Flatten()

        # Hesaplama: 224 -> 112 -> 56 -> 28 -> 14 (4 kez pooling yapıldı)
        # Son boyut: 256 kanal * 14 * 14
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.dropout = nn.Dropout(0.5) # Overfitting engellemek için
        self.fc2 = nn.Linear(512, num_classes) # Çıkış: 2 (Normal vs Pneumonia) [cite: 6]

    def forward(self, x):
        # Blok 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        # Blok 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        # Blok 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        # Blok 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))

        # Sınıflandırma
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Modeli Başlatma ve Cihaza Gönderme
model = SimpleCNN(num_classes=2).to(device)

# Model Özeti
print(model)

# Test Amaçlı Bir Geçiş (Forward Pass)
# Oluşturduğumuz batch'i modelden geçirip boyut kontrolü yapalım
with torch.no_grad():
    dummy_output = model(batch_img.to(device))
    print(f"\nModel Çıkış Boyutu: {dummy_output.shape}") # Beklenen: [32, 2]

In [ ]:
import time
from tqdm import tqdm # İlerleme çubuğu için

# 1. Hiperparametreler ve Ayarlar
LEARNING_RATE = 0.001
NUM_EPOCHS = 10 # Başlangıç için 10 epoch yeterli olacaktır

# Kayıp Fonksiyonu ve Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 2. Eğitim Fonksiyonu
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    since = time.time()

    # Metrikleri saklamak için listeler
    history = {
        'train_loss': [], 'train_acc': [],
        'test_loss': [], 'test_acc': []
    }

    print(f"Eğitim {device} üzerinde başlıyor...")
    print("-" * 30)

    for epoch in range(num_epochs):
        # --- TRAINING PHASE (Eğitim Aşaması) ---
        model.train() # Modeli eğitim moduna al (Dropout/BatchNorm aktif)
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # tqdm ile ilerleme çubuğu ekleyelim
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", unit="batch")

        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            # 1. Gradients sıfırla
            optimizer.zero_grad()

            # 2. Forward Pass (İleri yayılım)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 3. Backward Pass ve Optimize (Geri yayılım)
            loss.backward()
            optimizer.step()

            # İstatistikleri topla
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # Pbar güncelleme
            pbar.set_postfix({'loss': loss.item()})

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train

        # --- VALIDATION PHASE (Test Aşaması) ---
        model.eval() # Modeli değerlendirme moduna al
        running_test_loss = 0.0
        correct_test = 0
        total_test = 0

        with torch.no_grad(): # Gradyan hesaplama, bellek tasarrufu yap
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_test_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        epoch_test_loss = running_test_loss / len(test_loader.dataset)
        epoch_test_acc = correct_test / total_test

        # Geçmişe kaydet
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        history['test_loss'].append(epoch_test_loss)
        history['test_acc'].append(epoch_test_acc)

        print(f"Epoch {epoch+1}/{num_epochs} Sonucu:")
        print(f"Train Loss: {epoch_loss:.4f} | Train Acc: %{epoch_acc*100:.2f}")
        print(f"Test  Loss: {epoch_test_loss:.4f} | Test  Acc: %{epoch_test_acc*100:.2f}")
        print("-" * 30)

    time_elapsed = time.time() - since
    print(f'Eğitim tamamlandı! Toplam süre: {time_elapsed // 60:.0f}dk {time_elapsed % 60:.0f}sn')
    print(f'En iyi Test Doğruluğu: %{max(history["test_acc"])*100:.2f}')

    return history

# 3. Eğitimi Çalıştır
history = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)


In [ ]:
# Grafik çizdirme fonksiyonu
def plot_history(history):
    acc = history['train_acc']
    val_acc = history['test_acc']
    loss = history['train_loss']
    val_loss = history['test_loss']
    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # Doğruluk Grafiği
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Eğitim Doğruluğu (Train Acc)')
    plt.plot(epochs_range, val_acc, label='Test Doğruluğu (Test Acc)')
    plt.legend(loc='lower right')
    plt.title('Eğitim ve Test Doğruluğu')
    plt.grid(True)

    # Kayıp (Loss) Grafiği
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Eğitim Kaybı (Train Loss)')
    plt.plot(epochs_range, val_loss, label='Test Kaybı (Test Loss)')
    plt.legend(loc='upper right')
    plt.title('Eğitim ve Test Kaybı')
    plt.grid(True)

    plt.show()

plot_history(history)

In [ ]:
# ResNet50 ile devam
from torchvision import models
import torch.nn as nn

# 1. Önceden Eğitilmiş ResNet50 Modelini Yükle
# 'weights=models.ResNet50_Weights.IMAGENET1K_V1' parametresi, modelin ImageNet ağırlıklarıyla gelmesini sağlar.
print("ResNet50 Modeli İndiriliyor...")
weights = models.ResNet50_Weights.IMAGENET1K_V1
resnet_model = models.resnet50(weights=weights)

# 2. Modelin Son Katmanını (Classifier) Değiştirme
# ResNet50'nin orijinal çıkışı 1000 sınıftır (ImageNet). Biz bunu 2 sınıfa (Normal, Pneumonia) düşüreceğiz.
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 2)

# 3. Modeli GPU'ya Taşı
resnet_model = resnet_model.to(device)

print("\n--- Model Yapısı (Son Katman) ---")
print(resnet_model.fc) # Çıktı: Linear(in_features=2048, out_features=2, bias=True)

# 4. Optimizer ve Loss Ayarları
# Transfer Learning'de genelde öğrenme hızı (learning rate) biraz daha düşük tutulur.
resnet_optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.0001)
resnet_criterion = nn.CrossEntropyLoss()

# 5. Eğitimi Başlat (Mevcut train_model fonksiyonunu kullanıyoruz)
print("\nResNet50 Eğitimi Başlıyor...")
resnet_history = train_model(
    model=resnet_model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=resnet_criterion,
    optimizer=resnet_optimizer,
    num_epochs=5 # ResNet güçlü olduğu için 5 epoch bile çok iyi sonuç verebilir
)

In [ ]:
# Sonuçları çizdir
plot_history(resnet_history)

# En iyi doğruluk değerini yazdır
print(f"ResNet50 En İyi Test Doğruluğu: %{max(resnet_history['test_acc'])*100:.2f}")

In [ ]:
# Vision Transformer (ViT) Modeli

from transformers import ViTForImageClassification, ViTConfig

# 1. ViT Modelini Hazırlama
print("ViT Modeli (google/vit-base-patch16-224) İndiriliyor...")

# Etiket haritalaması (Raporlamada işe yarar)
id2label = {0: "NORMAL", 1: "PNEUMONIA"}
label2id = {"NORMAL": 0, "PNEUMONIA": 1}

# Modeli yükle
vit_model_hf = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True # Çıkış katmanını bizim 2 sınıfımıza göre yeniden boyutlandırır
)

# 2. Model Wrapper (Uyarlayıcı)
# Hugging Face modelleri 'output.logits' döndürür, bizim döngümüz direkt tensor bekliyor.
# Bu sınıf aradaki uyumu sağlar.
class ViTWrapper(nn.Module):
    def __init__(self, model):
        super(ViTWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        # ViT 'pixel_values' argümanı ister
        outputs = self.model(pixel_values=x)
        return outputs.logits

# Wrapper ile modeli sarıyoruz
vit_model = ViTWrapper(vit_model_hf)
vit_model = vit_model.to(device)

# 3. Optimizer Ayarları
# ViT için genelde learning rate biraz daha düşük tutulur (2e-5 veya 5e-5 gibi)
vit_optimizer = torch.optim.Adam(vit_model.parameters(), lr=5e-5)
vit_criterion = nn.CrossEntropyLoss()

# 4. Eğitimi Başlat
print("\nVision Transformer (ViT) Eğitimi Başlıyor...")
# ViT daha kompleks olduğu için yine 5 epoch yeterli olacaktır, hatta 3. epochta bile sonuç verebilir.
vit_history = train_model(
    model=vit_model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=vit_criterion,
    optimizer=vit_optimizer,
    num_epochs=5
)

In [ ]:
# ViT Sonuçlarını Çizdir
plot_history(vit_history)

# En iyi skoru yazdır
print(f"ViT En İyi Test Doğruluğu: %{max(vit_history['test_acc'])*100:.2f}")

In [ ]:

# --- ADIM 1: Veri Seti Ayarlarını Onarma ---
# Hafızadaki test_ds nesnesinin dönüşüm fonksiyonunu tekrar tanımlayıp atıyoruz.
# Bu işlem 'NoneType' hatasını ortadan kaldırır.

def apply_transforms(examples):
    examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["image"]]
    return examples

# Veri setini eski çalışan haline getiriyoruz
test_ds.set_transform(apply_transforms)
print("✅ Veri seti ayarları onarıldı.")


# --- ADIM 2: Grad-CAM Görselleştirme (Garantili Yöntem) ---
# Bu yöntem, ham görüntüye erişmek yerine elimizdeki Tensor'u
# matematiksel olarak geri çevirip görselleştirir. Hata riskini sıfıra indirir.

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import torch
import matplotlib.pyplot as plt

# ResNet50'nin son katmanına kanca atıyoruz
target_layers = [resnet_model.layer4[-1]]
cam = GradCAM(model=resnet_model, target_layers=target_layers)

# Tensor (Normalize edilmiş) -> Orijinal Görüntü (Görselleştirmek için)
def inverse_normalize(tensor):
    # (C, H, W) -> (H, W, C) formatına çevir
    img = tensor.permute(1, 2, 0).cpu().numpy()
    # Normalize işlemini geri al: (img * std) + mean
    # Daha önce mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5] kullanmıştık
    img = img * 0.5 + 0.5
    # 0 ile 1 arasına sıkıştır
    img = np.clip(img, 0, 1)
    return img

def visualize_gradcam_final(model, dataset, num_samples=4):
    model.eval()
    print("Pnömoni örnekleri üzerinde analiz yapılıyor...")

    # Rastgele pnömoni örnekleri bulalım
    found_samples = []
    indices = np.random.permutation(len(dataset)) # Rastgele tara

    for idx in indices:
        item = dataset[int(idx)]
        if item['label'] == 1: # Sadece PNEUMONIA
            found_samples.append(item)
            if len(found_samples) == num_samples:
                break

    plt.figure(figsize=(20, 6))

    for i, item in enumerate(found_samples):
        # Modelin beklediği input
        input_tensor = item['pixel_values'].unsqueeze(0).to(device)

        # 1. Grad-CAM (Isı Haritası) Hesapla
        grayscale_cam = cam(input_tensor=input_tensor, targets=None)
        grayscale_cam = grayscale_cam[0, :]

        # 2. Görüntüyü İnsan Gözü İçin Geri Çevir
        rgb_img = inverse_normalize(item['pixel_values'])

        # 3. Birleştir
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        # 4. Çizdir
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(visualization)
        plt.title("Modelin Karar Bölgesi\n(Kırmızı Alan)")
        plt.axis('off')

    plt.show()

# Çalıştır
visualize_gradcam_final(resnet_model, test_ds, num_samples=4)

In [ ]:
import os

# Kayıt yolu
save_path = '/content/drive/MyDrive/Colab_Notebooks/Y.Lisans Proje Odevi/Models'
os.makedirs(save_path, exist_ok=True)

# ResNet Modelini Kaydet
torch.save(resnet_model.state_dict(), os.path.join(save_path, 'resnet50_pneumonia.pth'))

# ViT Modelini Kaydet
torch.save(vit_model.state_dict(), os.path.join(save_path, 'vit_pneumonia.pth'))

print(f"Modeller {save_path} klasörüne kaydedildi.")

In [ ]:
import os
import torch
import pickle

# 1. Kayıt Klasörünü Belirleme
# Proje klasörünüzün altına 'Model_Outputs' adında bir klasör açalım
project_path = '/content/drive/MyDrive/Colab_Notebooks/Y.Lisans Proje Odevi'
save_path = os.path.join(project_path, 'Model_Outputs')

# Klasör yoksa oluştur
os.makedirs(save_path, exist_ok=True)
print(f"📂 Kayıt klasörü hazırlandı: {save_path}")

# --- 2. Modelleri Kaydetme (.pth dosyaları) ---
print("\n💾 Modeller kaydediliyor...")

# Baseline CNN
if 'model' in locals():
    torch.save(model.state_dict(), os.path.join(save_path, 'baseline_cnn.pth'))
    print("✅ Baseline CNN kaydedildi.")

# ResNet50
if 'resnet_model' in locals():
    torch.save(resnet_model.state_dict(), os.path.join(save_path, 'resnet50.pth'))
    print("✅ ResNet50 kaydedildi.")

# Vision Transformer (ViT)
if 'vit_model' in locals():
    torch.save(vit_model.state_dict(), os.path.join(save_path, 'vit_base.pth'))
    print("✅ ViT Model kaydedildi.")

# --- 3. Eğitim Geçmişini Kaydetme (Grafikler için veriler) ---
print("\n📊 Eğitim geçmişleri (History) kaydediliyor...")

histories = {}
if 'history' in locals(): histories['cnn'] = history
if 'resnet_history' in locals(): histories['resnet'] = resnet_history
if 'vit_history' in locals(): histories['vit'] = vit_history

# Pickle modülü ile sözlük yapısını dosyaya döküyoruz
with open(os.path.join(save_path, 'training_histories.pkl'), 'wb') as f:
    pickle.dump(histories, f)

print(f"✅ Tüm eğitim geçmişleri 'training_histories.pkl' dosyasına kaydedildi.")
print("\n🎉 Tüm işlem tamamlandı! Google Drive'ınızı kontrol edebilirsiniz.")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pandas as pd

# 1. Tahmin Fonksiyonu (Tüm modeller için ortak)
def get_all_predictions(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    # Tensörleri CPU'ya alıp listeye ekleyeceğiz
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

# 2. Matris Çizdirme ve Kaydetme Fonksiyonu
def plot_and_save_cm(model, model_name, loader, classes, save_folder):
    print(f"\n--- {model_name} Analizi Başlıyor ---")

    # Tahminleri al
    y_true, y_pred = get_all_predictions(model, loader, device)

    # Matrisi Hesapla
    cm = confusion_matrix(y_true, y_pred)

    # Raporu Yazdır (Precision, Recall, F1-Score)
    print(classification_report(y_true, y_pred, target_names=classes))

    # Görselleştirme
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.ylabel('Gerçek Etiket (True Label)')
    plt.xlabel('Tahmin Edilen (Predicted Label)')
    plt.title(f'Confusion Matrix - {model_name}')

    # Kaydet
    file_path = os.path.join(save_folder, f'confusion_matrix_{model_name}.png')
    plt.savefig(file_path)
    plt.show()
    print(f"✅ Grafik kaydedildi: {file_path}")

# --- 3. Çalıştırma Kısmı ---

# Sınıf isimleri (Dokümana göre: 0=NORMAL, 1=PNEUMONIA)
class_names = ['NORMAL', 'PNEUMONIA']
save_path = '/content/drive/MyDrive/Colab_Notebooks/Y.Lisans Proje Odevi/Model_Outputs'

# a) Baseline CNN
if 'model' in locals():
    plot_and_save_cm(model, "Baseline_CNN", test_loader, class_names, save_path)

# b) ResNet50
if 'resnet_model' in locals():
    plot_and_save_cm(resnet_model, "ResNet50", test_loader, class_names, save_path)

# c) Vision Transformer (ViT)
if 'vit_model' in locals():
    plot_and_save_cm(vit_model, "ViT_Base", test_loader, class_names, save_path)